In [ ]:
from mysql.connector import connect

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
from ssl import create_default_context, CERT_NONE
import json
import pandas as pd
import datetime
import logging
import re

import psycopg2
import pandas as pd


# DB_NAMES = ["edm", "segdb", "campaign","accounts"]

DB_USER = "postgres"
DB_PASS = "infopower"
DB_HOST = "localhost"  # or the IP address of your database server
DB_PORT = "9100"       # default PostgreSQL port

# Connect to your PostgreSQL database
connection = psycopg2.connect(
    dbname='vector_db',
    user=DB_USER,
    password=DB_PASS,
    host=DB_HOST,
    port=DB_PORT
)
cursor = connection.cursor()

# Execute a SELECT query
cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public' and table_name != 'goose_db_version' 
            """)

result = cursor.fetchall()

result_table = cursor.execute(f"SELECT {result[0][0]} FROM vector_db LIMIT 5;")
result_table = cursor.fetchall()
result_table_df = pd.DataFrame(result_table)
result_table_df.head()

# ES query

In [1]:
from mysql.connector import connect
from elasticsearch import  helpers
import pandas as pd
import datetime

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
sys.path.append(r'D:\markding_git\big-data-ai-integration-platform\search_system\FastAPI_service\code')
from connection import ElasticSearchConnectionManager


In [2]:
# es = create_es_connection()
es = ElasticSearchConnectionManager.get_instance()

print(es.info())

es_health = es.health_report()
print(es_health['status'])
indicators = es_health['indicators']
[f"status_of_[{i}] : {indicators[i]['status']}" for i in indicators.keys()]

('elastic', 'gAcstb8v-lFCVzCBC__a')
{'name': '7d664ea53c15', 'cluster_name': 'docker-cluster', 'cluster_uuid': '0XOBCloyQGW6WDdj1nbg3Q', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
yellow


['status_of_[master_is_stable] : green',
 'status_of_[repository_integrity] : green',
 'status_of_[disk] : green',
 'status_of_[shards_capacity] : green',
 'status_of_[shards_availability] : yellow',
 'status_of_[slm] : green',
 'status_of_[ilm] : green']

In [9]:
response = es.search(index='whole_corp', body= {
      "aggs": {
        "duplicate_docs": {
          "terms": {
            "field": "統一編號.keyword",
            "min_doc_count": 2 #// Find terms with more than one document
          },
          "aggs": {
            "top_duplicate_hits": {
              "top_hits": {
                "size": 100 #// Adjust size as needed to retrieve all duplicate IDs
              }
            }
          }
        }
      }
    }   
)
list_ = pd.DataFrame([hit['_source'] for hit in response['hits']['hits']])
list_


,統一編號,公司名稱,負責人,登記地址,資本額,營業項目及代碼表,縣市名稱,區域名稱,縣市區域,類別_全,官網,電話
0,94228520,錦象科股份有限公司,楊O寧,臺中市南區學府路168巷7號11樓之12,5000000,"園藝服務業,造林業,伐木業,其他動物服務業,室內裝潢業,油漆工程業,球場跑道樹脂材料鋪設工程...",臺中市,南區,台中市南區,"園藝服務業,造林業,伐木業,其他動物服務業,室內裝潢業,油漆工程業,球場跑道樹脂材料鋪設工程...",\N,\N\r
1,94231308,沐捷有限公司,李O臻,高雄市大寮區中庄路73號,1000000,"園藝服務業,國際貿易業,室內裝潢業,花卉批發業,花卉零售業,五金批發業,人力派遣業,藝文服務...",高雄市,大寮區,高雄市大寮區,"園藝服務業,國際貿易業,室內裝潢業,花卉批發業,花卉零售業,五金批發業,人力派遣業,藝文服務...",\N,\N\r
2,94231953,禾豐園藝有限公司,胡靜玲,高雄市仁武區澄合街628號1樓,500000,"農作物栽培業,特用作物栽培業,食用菌菇類栽培業,花卉栽培業,園藝服務業,造林業,種苗批發業,...",高雄市,仁武區,高雄市仁武區,"農作物栽培業,特用作物栽培業,食用菌菇類栽培業,花卉栽培業,園藝服務業,造林業,種苗批發業,...",\N,\N\r
3,94240926,昇霖景觀有限公司,張O正,新北市土城區金城路3段207巷6弄10號3樓,5000000,"園藝服務業,室內設計業,花卉批發業,花卉零售業,花藝設計業,其他設計業",新北市,土城區,新北市土城區,"園藝服務業,室內設計業,花卉批發業,花卉零售業,花藝設計業,其他設計業",\N,\N\r
4,94241657,太一園藝有限公司,吳O原,新北市金山區中信北路30號,1000000,"室內設計業,園藝服務業,室內裝潢業,油漆工程業,照明設備安裝工程業,門窗安裝工程業",新北市,金山區,新北市金山區,"室內設計業,園藝服務業,室內裝潢業,油漆工程業,照明設備安裝工程業,門窗安裝工程業",\N,\N\r
5,94251057,萌美學有限公司,劉O均,新北市三重區新北大道1段123號11樓,500000,"園藝服務業,花卉批發業,裝設品批發業,陶瓷玻璃器皿批發業,五金批發業,育樂用品批發業,育樂用...",新北市,三重區,新北市三重區,"園藝服務業,花卉批發業,裝設品批發業,陶瓷玻璃器皿批發業,五金批發業,育樂用品批發業,育樂用...",\N,\N\r
6,94252680,潮州綠藝環保有限公司,邱瑞文,屏東縣潮州鎮三星里7鄰壽福路116號1樓,3500000,"病媒防治業,廢棄物清除業,廢棄物處理業,花卉栽培業,花卉零售業,其他批發業,花藝設計業,其他...",屏東縣,潮州鎮,屏東縣潮州鎮,"病媒防治業,廢棄物清除業,廢棄物處理業,花卉栽培業,花卉零售業,其他批發業,花藝設計業,其他...",\N,\N\r
7,94271130,石庭園藝有限公司,徐O勝,新北市土城區國際路158巷26號,1000000,"園藝服務業,農作物栽培業,花卉栽培業,農產品整理業,作物栽培服務業,花卉批發業,花卉零售業,...",新北市,土城區,新北市土城區,"園藝服務業,農作物栽培業,花卉栽培業,農產品整理業,作物栽培服務業,花卉批發業,花卉零售業,...",\N,\N\r
8,94289407,甄鴻企業社,方若瑜,屏東縣內埔鄉內田村自強路11號2樓,200000.0,"花卉栽培業,園藝服務業,花卉批發業,五金批發業,建材批發業,機車零件配備批發業,國際貿易業,...",屏東縣,內埔鄉,屏東縣內埔鄉,"花卉栽培業,園藝服務業,花卉批發業,五金批發業,建材批發業,機車零件配備批發業,國際貿易業,...",\N,\N\r
9,94291430,木木貓企業社,林恉安,屏東縣里港鄉過江村過江路119之6號1樓,200000.0,"農作物栽培業,肥料零售業,日常用品零售業,育樂用品零售業,污染防治設備零售業,首飾及貴金屬零...",屏東縣,里港鄉,屏東縣里港鄉,"農作物栽培業,肥料零售業,日常用品零售業,育樂用品零售業,污染防治設備零售業,首飾及貴金屬零...",\N,\N\r


In [10]:
list_.columns

Index(['統一編號', '公司名稱', '負責人', '登記地址', '資本額', '營業項目及代碼表', '縣市名稱', '區域名稱',
       '縣市區域', '類別_全', '官網', '電話'],
      dtype='object')

In [37]:
# Extract whole data from ES

In [19]:
index_name = "whole_corp"
scroll_time = "2m"   # keep context alive
batch_size = 1000

response = es.search(
    index=index_name,
    scroll=scroll_time,
    size=batch_size,
    body = {
              "aggs": {
        "duplicate_docs": {
          "terms": {
            "field": "統一編號.keyword",
            "min_doc_count": 2 #// Find terms with more than one document
          },
          "aggs": {
            "top_duplicate_hits": {
              "top_hits": {
                "size": 100 #// Adjust size as needed to retrieve all duplicate IDs
              }
            }
          }
        }
      }
    }
)

scroll_id = response['_scroll_id']
hits = response["hits"]["hits"]

all_hits = []
all_hits.extend(hits)

while len(hits) > 0:
    response = es.scroll(scroll_id=scroll_id, scroll=scroll_time)
    scroll_id = response["_scroll_id"]
    hits = response["hits"]["hits"]
    all_hits.extend(hits)

print(f"Total docs: {len(all_hits)}")

Total docs: 1281019


In [7]:
for index_name in ['whole_corp']:
    try:
        print(es.count(index=index_name.lower()))
    except Exception as e:
        print(f'\n==============no values in index name [{index_name}]================')
# es.close()

{'count': 1113529, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}


# Recreate index and delete olde index

In [29]:
sys.path.append(r'D:\markding_git\big-data-ai-integration-platform\airflow\dags\code')
from es_mapping import corp_mapping

In [32]:
src_index = "whole_corp"
dst_index = "whole_corp_dedup"

# 1. Create target index with same mapping
mapping = es.indices.get_mapping(index=src_index)
es.indices.create(index=dst_index, body=corp_mapping())

# 2. Stream and deduplicate
seen = set()

def dedup_stream():
    for doc in helpers.scan(es, index=src_index, query={"query": {"match_all": {}}}, size=1000):
        uid = doc["_source"]["統一編號"]
        if uid not in seen:
            seen.add(uid)
            yield {
                "_op_type": "index",
                "_index": dst_index,
                "_id": uid,              # force uniqueness on 統一編號
                "_source": doc["_source"]
            }

helpers.bulk(es, dedup_stream(), chunk_size=2000)

print("Dedup index created.")


Dedup index created.


# Delete old one and alias 

In [34]:
es.indices.delete(index='whole_corp')

ObjectApiResponse({'acknowledged': True})

In [5]:
es.indices.put_alias(index='whole_corp_dedup', name='whole_corp')

ObjectApiResponse({'acknowledged': True})

In [6]:
es.indices.get_alias(index='whole_corp_dedup')


ObjectApiResponse({'whole_corp_dedup': {'aliases': {'whole_corp': {}}}})

In [ ]:
response = es.search(index='whole_corp',body={
    "query": {"match_all": {}},
    "_source": ["cluster","vector"],
    "size": 1
}

)
df_ = pd.DataFrame([hit['_source'] for hit in response['hits']['hits']])
df_


""
0
1
2
3
4
5
6
7
8
9


In [28]:
response = es.search(index='whole_corp',body={'size': 30, 'query': {'script_score': {'query': {'term': {'cluster': '6'}}, 'script': {'source': "\n                        double cos = cosineSimilarity(params.qv, 'vector') + 1.0;\n                        return cos;\n                    ", 'params': {'qv': [0.01015218294295369, 0.022547749165193236, 0.014298604664889374, 0.026228218743844588, 0.031219150957289853, 0.043686774434826346, 0.03693182587343318, 0.0474278399487247, 0.0160172750481279, 0.06394944110272957, 0.04510127523955598, 0.04529989972066639, 0.10545495509916972, -0.025880042174206412, 0.05633546725392328, 0.018601243744636058, -0.039133795436959856, 0.0019299609052261665, -0.0333257534333357, 1.8450152280709544, -0.0065837085199924456, 0.016881133219277106, 0.012822248278660011, 0.0457910086257529, 0.009043665401400816, 0.04899175727740487, -0.06423218946358106, -0.09313359903984392, 0.014642119101468962, -0.0010734168357718309, -0.00877245942347962, 0.019933511468276367, 0.00549284967346186, -0.00982347959649498, -0.014187422717083092, -0.005438113922469025, 0.06034141304098674, 0.034425313721851966, 0.054973762042401064, 0.010525447543249068, 0.00924111888202848, 0.015930823580685528, -0.06291726017215381, -0.03924596481209802, -0.007777771863849153, 0.118272682371092, 0.008627461781745498, 0.0371871556976551, 0.05180579427041952, 0.06345095264977416, 0.02611537995602823, 0.02952413806824032, 0.05793069481039696, -0.014455362658981306, 0.040064271387765096, 0.04787881494122987, 0.09191532815813952, 0.03789914060147712, -0.03551099952376034, -0.032414935000982026, 0.02055804223150503, 0.07768513369648702, 0.11373028319450014, -0.01991036214301229, 0.06164715783600624, 0.06527513245650754, -0.01983726921001074, 0.0002924608887871494, 0.08099897944129916, 0.0650880420758927, -0.06890653773525043, 0.03971820169811278, 0.08766815419091248, 0.03635913997800704, 0.030339897897018447, -0.020469145200618, -0.03771297724364999, -0.020605308500586134, 0.013964837384248304, -0.10817816265069014, 0.059526436787793215, 0.05270268776229572, 0.04450930658136586, 0.07534388135555446, -0.0011991416482456038, -0.05172842384945537, 0.06141170743542297, 0.05313193034398131, -0.1011723602480556, -0.02776267023126863, 0.08622164296745286, -0.03136187306051941, -0.0676227526177238, 0.13539670753302446, 0.0017864230680693337, 0.007435176598968637, 0.07935812006942242, 0.0999149398540881, -0.06082050879801946, 0.07570757744098147]}}}}}

)
df_ = pd.DataFrame([hit['_source'] for hit in response['hits']['hits']])
df_


,統一編號,公司名稱,負責人,登記地址,資本額,營業項目及代碼表,縣市名稱,區域名稱,縣市區域,類別_全,官網,電話,cluster,vector
0,01189092,啟延電子科技股份有限公司,\N,臺北市內湖區內湖路1段306號6樓之2,198000000,"資料處理服務, 資訊軟體服務, 電子資訊供應服務業",臺北市,內湖區,臺北市內湖區,"資料處理服務, 資訊軟體服務, 電子資訊供應服務業",https://119575.web66.tw\n,02-87975969\r,6,"[0.01015218294295369, 0.022547749165193236, 0...."
1,01518051,京威企業股份有限公司,馬O芬,臺北市松山區民生東路4段54號10樓之8,29500000,"國際貿易業, 日常用品批發業, 食品什貨批發業",臺北市,松山區,臺北市松山區,"國際貿易業, 日常用品批發業, 食品什貨批發業",\N,\N\r,6,"[0.00919367767332829, 0.02044965509724381, 0.0..."
2,01188170,皓宇企業股份有限公司,侯O仲,臺北市南港區南港路3段50巷1號4樓,10000000,"五金批發業, 國際貿易業, 資訊軟體服務, 電器批發業",臺北市,南港區,臺北市南港區,"五金批發業, 國際貿易業, 資訊軟體服務, 電器批發業",\N,\N\r,6,"[0.008813630260719851, 0.01971269220543506, 0...."
3,01515619,德兆建設股份有限公司,王O德,臺北市大安區仁愛路4段376號8樓,400000000,"一般廣告服務業, 五金批發業, 國際貿易業, 室內裝潢, 建材批發業",臺北市,大安區,臺北市大安區,"一般廣告服務業, 五金批發業, 國際貿易業, 室內裝潢, 建材批發業",\N,\N\r,6,"[0.01087265463316192, 0.02433175535367116, 0.0..."
4,01186378,鎂得有限公司,汪O攻,新北市三重區重新路5段609巷6號6樓之3,5000000,"五金批發業, 國際貿易業, 機械批發業, 電器批發業",新北市,三重區,新北市三重區,"五金批發業, 國際貿易業, 機械批發業, 電器批發業",\N,\N\r,6,"[0.00852341115805104, 0.019110045506611772, 0...."
5,01518812,振遠科技股份有限公司,許O仁,臺北市南港區經貿二路189號10樓,3500000000,"國際貿易業, 產品設計業, 資料處理服務, 資訊軟體服務, 電子資訊供應服務業",臺北市,南港區,臺北市南港區,"國際貿易業, 產品設計業, 資料處理服務, 資訊軟體服務, 電子資訊供應服務業",\N,\N\r,6,"[0.0120152932898907, 0.026877326326269105, 0.0..."
6,01456180,三力電腦股份有限公司,蔡O基,臺北市大安區敦化南路1段200號7樓之4,8000000,"其他工商服務業, 國際貿易業, 資料處理服務, 資訊軟體服務",臺北市,大安區,臺北市大安區,"其他工商服務業, 國際貿易業, 資料處理服務, 資訊軟體服務",\N,\N\r,6,"[0.008738830641546897, 0.019560898868558635, 0..."
7,00803985,誼信機械工程有限公司,王O偉,新北市金山區中正路141巷1號3樓,5000000,"人力派遣業, 環保服務, 電器零售業",新北市,金山區,新北市金山區,"人力派遣業, 環保服務, 電器零售業",\N,\N\r,6,"[0.00837236056608485, 0.0187300204039949, 0.01..."
8,01459124,國陞工程企業有限公司,陳O燕,臺北市中正區寧波西街87號2樓之2,5000000,"化粧品零售業, 國際貿易業, 日常用品零售業, 食品什貨、飲料零售業",臺北市,中正區,臺北市中正區,"化粧品零售業, 國際貿易業, 日常用品零售業, 食品什貨、飲料零售業",\N,\N\r,6,"[0.008304724105712594, 0.01851501709779245, 0...."
9,00842404,富國營造有限公司,張O懋,新北市金山區中正路79號,12000000,"人力派遣業, 環保服務, 門窗安裝工程業",新北市,金山區,新北市金山區,"人力派遣業, 環保服務, 門窗安裝工程業",\N,\N\r,6,"[0.0087746737596196, 0.01959711156055358, 0.01..."


In [20]:
response = es.search(
    index="whole_corp_dedup",
    body={
        "query": {"match_all": {}},
        "_source": ["cluster","vector"],
        "size": 1
    }
)
print(response["hits"]["total"])
print(response["hits"]["hits"])


{'value': 10000, 'relation': 'gte'}
[{'_index': 'whole_corp_dedup', '_id': '94228520', '_score': 1.0, '_source': {}}]


In [19]:
response['hits']['hits']

[{'_index': 'whole_corp_dedup',
  '_id': '94228520',
  '_score': 1.0,
  '_source': {}}]

In [8]:
df_

NameError: name 'df_' is not defined